In [66]:
import numpy as np
import pandas as pd 
from PIL import Image
import os

import torch
import torch.nn as nn
import torchvision.transforms as transform
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, ConcatDataset, Subset

from tqdm import tqdm
import random

#### set random seed

In [67]:
def setSeed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

#### transform

In [68]:
test_tfm = transform.Compose([
    transform.Resize((128, 128)),
    transform.ToTensor(),
    transform.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_tfm = transform.Compose([
    transform.RandomRotation(40),
    transform.RandomAffine(degrees=0, translate=(0.2, 0.2), shear=0.2),
    transform.RandomHorizontalFlip(p=0.5),
    transform.Resize((128, 128)),
    transform.ToTensor(),
    transform.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#### dataset

In [69]:
class foodDataset(Dataset):

    def __init__(self, path, tfm=test_tfm):
        super(foodDataset).__init__()
        self.path = path
        self.tfm = tfm
        self.imgName = sorted([name for name in os.listdir(self.path) if name.endswith('.jpg')])
        self.imgPath = [os.path.join(self.path, name) for name in self.imgName]
     
    def __getitem__(self, idx):
        img = Image.open(self.imgPath[idx])
        img = self.tfm(img)
        try:
            label = int(self.imgName[idx].split('_')[0])
        except:
            label = -1
        return img, label
    
    def __len__(self):
        return len(self.imgName)


#### model

In [70]:
class cnnBlock(nn.Module):

    def __init__(self, input_chann, output_channel, kernel_size=3, stride=1, padding=1):
        super(cnnBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(input_chann, output_channel, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(output_channel),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)
        )

    def forward(self, x):
        return self.block(x)

In [71]:
class linearBlock(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(linearBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.Dropout(0.4),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.block(x)

In [72]:
class foodClassifier(nn.Module):
    
    def __init__(self, cnnLayers, linearLayers):
        super(foodClassifier, self).__init__()
        self.cnn = nn.Sequential(
            *[cnnBlock(cnnLayers[i-1], cnnLayers[i]) for i in range(1, len(cnnLayers))]
        )
        self.linear = nn.Sequential(
            *[linearBlock(linearLayers[i-1], linearLayers[i]) for i in range(1, len(linearLayers))]
        )
    
    def forward(self, x):
        x = self.cnn(x)
        x = x.flatten(start_dim=1)
        x = self.linear(x)
        return x
        

#### Pseudo

In [73]:
class PseudoDataset(Dataset):

    def __init__(self, unlabeled_set, indices, pseudo_labels):
        self.data = Subset(unlabeled_set, indices)
        self.target = torch.LongTensor(pseudo_labels)[indices]

    def __getitem__(self, index):

        if index < 0:
            index += len(self)
        if index > len(self):
            raise IndexError("index %d is out of bounds for axis 0 with size %d"%(index, len(self)))
        
        x = self.data[index][0]
        y = self.target[index].item()

        return x, y
    
    def __len__(self):
        return len(self.data)

In [74]:
def get_pseudo_dataset(unlabeled_dataset, model, config, threshhold=0.65):

    print('get pseudo labels ... ')
    masks = []
    pseudo_labels = []
    model.eval()

    data_loader = DataLoader(unlabeled_dataset, batch_size=32, shuffle=False)

    for batch in tqdm(data_loader):
        img, _ = batch
        img = img.to(config['device'])

        with torch.no_grad():
            pred = model(img)
            pred = torch.softmax(pred, dim=1).cpu()

        index = torch.argmax(pred, dim=1)
        giveup = torch.max(pred, dim=1)[0] > threshhold
        pseudo_labels.append(index)
        masks.append(giveup)
    
    pseudo_labels = torch.cat(pseudo_labels, dim=0)
    masks = torch.cat(masks, dim=0)
    indices = torch.arange(0, len(unlabeled_dataset))[masks]
    pseudo = PseudoDataset(unlabeled_dataset, indices, pseudo_labels)
    print(f'use {len(indices)/len(unlabeled_dataset):.2f} unlabeled data')

    return pseudo            


#### Trainer

In [75]:
def Trainer(model, train_set, valid_set, config, unlabeled_set=None):
    model.to(config['device'])
    train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
    valid_loader = DataLoader(valid_set, batch_size=config['batch_size'])

    optimizer = torch.optim.Adam(model.parameters(), config['learning_rate'], weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma'])
    criterion = torch.nn.CrossEntropyLoss()

    best_acc = 0
    step = 0

    writer = SummaryWriter()

    for epoch in range(config['epochNum']):

        if config['semi-supervise']:
            pseudoDataset = get_pseudo_dataset(unlabeled_set, model, config, threshhold=0.85)
            train_set = ConcatDataset((pseudoDataset, train_set))
            train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
            print("total number of training data: ", len(train_set))

        model.train()

        train_loss, train_acc = 0.0, 0.0

        for data in tqdm(train_loader):
            img, label = data
            img, label = img.to(config['device']), label.to(config['device'])

            pred = model(img)
            loss = criterion(pred, label)

            # clear gradients in every batch
            optimizer.zero_grad()

            # grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            loss.backward()

            optimizer.step()

            train_acc += (pred.argmax(dim=-1) == label).float().mean()
            train_loss += loss.item()

        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)
        writer.add_scalar(config['writerName'] + 'Loss/Train', train_loss, epoch)
        writer.add_scalar(config['writerName'] + 'Accuracy/Train', train_acc, epoch)

        model.eval()
        valid_acc, valid_loss = 0.0, 0.0
        for data in tqdm(valid_loader):
            img, label = data 
            img, label = img.to(config['device']), label.to(config['device'])
    
            with torch.no_grad():
                pred = model(img)

            loss = criterion(pred, label)

            valid_loss += loss.item()
            valid_acc += (pred.argmax(dim=-1) == label).float().mean()

        valid_acc, valid_loss = valid_acc / len(valid_loader), valid_loss / len(valid_loader)
        writer.add_scalar(config['writerName'] + 'Loss/Valid', valid_loss, epoch)
        writer.add_scalar(config['writerName'] + 'Accuracy/Valid', valid_acc, epoch)
        print(f'epoch[{epoch}] | train loss: {train_loss:.5f} train acc: {train_acc:.4f} | valid loss: {valid_loss:.5f} valid acc: {valid_acc:.4f}')

        scheduler.step()

        torch.save(model.state_dict(), os.path.join(config['modelSavePath'], config['lastModel']))
        if best_acc < valid_acc:
            best_acc = valid_acc
            torch.save(model.state_dict(), os.path.join(config['modelSavePath'], config['bestModel']))
            print(f'find A better model! acc: {best_acc:.4f}')
            step = 0
        else:
            step += 1
            if step > config['early_stop']:
                print('Cannot improve model~')
                break
    writer.close()


#### parameters

In [76]:
config = {
    'learning_rate': 1e-3,
    'batch_size': 64,
    'cnnLayers': [3, 64, 128, 256, 512],
    'linearLayers': [512*8*8, 1024, 512, 256, 11],
    'gamma': 0.8,
    'step_size': 10,
    'weight_decay': 1e-3,
    'seed': 914122,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'epochNum': 400,
    'early_stop': 50,
    'modelSavePath': './model/',
    'bestModel': 'best_0915.ckpt',
    'lastModel': 'last_0915.ckpt',
    'trainImgPath': './data/training/',
    'validImgPath': './data/validation/',
    'testImgPath': './data/test/',
    'writerName': 'HW3 ',
    'unlabelPath': './data/test',
    'semi-supervise': True
}

In [77]:
print(config['device'])

cpu


#### data & model

In [78]:
train_set = foodDataset(config['trainImgPath'], tfm=train_tfm)
valid_set = foodDataset(config['validImgPath'])
test_set = foodDataset(config['testImgPath'])
unlabel_set = foodDataset(config['unlabelPath'], tfm=train_tfm)

In [79]:
test_loader = DataLoader(test_set, batch_size=config['batch_size'])

In [80]:
model = foodClassifier(config['cnnLayers'], config['linearLayers'])
# model.load_state_dict(torch.load('./model/last_0915.ckpt'))

In [81]:
Trainer(model, train_set, valid_set, config, unlabeled_set=unlabel_set)

get pseudo labels ... 


100%|██████████| 1/1 [00:00<00:00,  6.89it/s]


use 0.00 unlabeled data
total number of training data:  33


100%|██████████| 1/1 [00:00<00:00,  2.19it/s]


epoch[0] | train loss: 2.77232 train acc: 0.1212 | valid loss: 2.39610 valid acc: 0.1333
find A better model! acc: 0.1333
get pseudo labels ... 


100%|██████████| 1/1 [00:00<00:00,  7.64it/s]


use 0.00 unlabeled data
total number of training data:  33


  0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 